In [12]:
#requires NLTK, BeautifulSoup
import load_data

import glob, os
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Embedding, SimpleRNN, Dropout
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing import sequence
from keras.utils import to_categorical

tk = Tokenizer(num_words=1000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True, split=" ", char_level=False)

labels = {"Obama" : 0, "Lincoln": 1, "Trump": 2}

input_text = []
input_labels = []
directory = "../data/processed/"
for filename in glob.glob(os.path.join(directory, '*.txt')):
    arr = filename.replace(directory,'').split("_")
    input_labels = input_labels + [labels[arr[0]]]
    input_text = input_text + [open(filename).read().decode("UTF-8").encode("ascii","ignore")] 
        
tk.fit_on_texts(input_text)
tk_test = tk.texts_to_sequences(input_text)

max_len = 1500
x = sequence.pad_sequences(tk_test, maxlen=max_len)
y = to_categorical(input_labels)

max_features = 20000

model = Sequential()
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='SGD',metrics=['categorical_accuracy'])
print(model.summary())

model.fit(x, y=y, batch_size=50, nb_epoch=10, verbose=1, validation_split=0.2, shuffle=True)



Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1500, 128)         2560000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1500, 128)         0         
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 387       
_________________________________________________________________
activation_8 (Activation)    (None, 3)                 0         
Total params: 2,593,283
Trainable params: 2,593,283
Non-trainable params: 0
_________________________________________________________________
None
Train on 34 samples, validate on 9 samples
Epoch 1/10
34/34 [==============================] - 1s - loss: 1.1227